# NYC Yellow cab dataset -- Preprocessing

This notebook is provided to generate file `yellow.csv`. This file is already provided in the _data/_ folder. 

`yellow` contains the number of taxis hired for each `taxi_zone` at a hourly time for the whole year 2017. 

The `yellow` dataset is computed from the ___NYC Taxi and Limousine Commission (LTC)___ [available data](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml).

In [1]:
import pandas as pd
import numpy as np
from time import time
from multiprocessing import Pool

## 1. Download Data

Only download the `TARGET_COLUMNS` from the S3 repository. 

__Warning:__ Even with an excellent connection, it takes around __10 min__ to download the full year.

In [2]:
URL = "https://s3.amazonaws.com/nyc-tlc/trip+data/"

TARGET_COLUMNS = ['tpep_pickup_datetime', 'PULocationID'] 

def get_month_dataset(month):
    url = URL + "yellow_tripdata_2017-{:0>2}.csv".format(month)
    print('--| ' + url)
    now = time()
    month_df = pd.read_csv(url, usecols=TARGET_COLUMNS)
    print(month, '-->', int(time()-now), 'seconds\n')
    return month_df

In [3]:
%%time

p = Pool(5)
months = list(p.map(get_month_dataset, range(1,13)))

--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-01.csv
--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-04.csv
--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-05.csv
--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-03.csv
--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-02.csv
5 --> 56 seconds

--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-06.csv
1 --> 87 seconds

--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-07.csv
3 --> 91 seconds

--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-08.csv
4 --> 103 seconds

--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-09.csv
2 --> 126 seconds

--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-10.csv
6 --> 131 seconds

--| https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-11.csv

8 --> 96 seconds
--| https://s3.amazonaws.com/nyc-tlc/trip+data

## 2. Data Preprocessing

### 2.1 Build a single DataFrame

Merge `months` into a single DataFrame: `yellow`.

In [4]:
months = pd.concat(months, ignore_index=True)

months.columns = ['pickup_datetime', 'taxi_zone']

### 2.2 Create an hourly timestamp

Truncate `pickup_datetime` to groupby every trips that are starting from the same `taxi_zone` at the same hourly time.

In [5]:
# Convert to datetime
months.pickup_datetime = pd.to_datetime(months.pickup_datetime, utc=True)

# Truncate the datetime to groupby by hour each trip
months.pickup_datetime = months.pickup_datetime.dt.floor('h')

# Add column to count the number of trips in each taxi zone
months['trip_counter'] = np.ones(months.shape[0])
months = months.groupby(['pickup_datetime', 'taxi_zone']).sum()

### 2.3 Index on time with NYC timezone
Set `pickup_datetime` as index.

In [6]:
# Set NYC timezone
months.reset_index(inplace=True)
months.set_index('pickup_datetime', inplace=True)
months.index = months.index.tz_convert('America/New_York')

### 2.4 Last cleaning step
Filter unwanted lines added retrospectively.

In [7]:
months = months['2017-01-01 00:00':'2017-12-31 23:00']

## 3. Reformat Dataframe

In [8]:
nyc_zones = sorted(months.taxi_zone.unique())

TIME_INDEX = pd.date_range("2017-01-01 00:00", "2017-12-31 23:00", freq="h", tz='America/New_York')

yellow = pd.DataFrame(
    data=['-05:00' for t in range(len(TIME_INDEX))],
    index=TIME_INDEX,
    columns=['timezone']
)

for z in nyc_zones:
    zone = months[months.taxi_zone==z].trip_counter
    zone = pd.DataFrame({z:zone})
    yellow = yellow.join(zone, how='left')

In [9]:
yellow.fillna(0, inplace=True)

yellow.head()

,timezone,1,2,3,4,5,6,7,8,9,...,256,257,258,259,260,261,262,263,264,265
2017-01-01 00:00:00-05:00,-05:00,2.0,0.0,0.0,54.0,0.0,0.0,66.0,0.0,0.0,...,50.0,1.0,0.0,2.0,49.0,23.0,35.0,133.0,135.0,13.0
2017-01-01 01:00:00-05:00,-05:00,2.0,0.0,0.0,36.0,0.0,0.0,54.0,1.0,0.0,...,30.0,0.0,4.0,2.0,31.0,17.0,49.0,106.0,86.0,10.0
2017-01-01 02:00:00-05:00,-05:00,2.0,0.0,0.0,28.0,0.0,0.0,42.0,0.0,0.0,...,6.0,0.0,0.0,0.0,16.0,14.0,72.0,92.0,80.0,6.0
2017-01-01 03:00:00-05:00,-05:00,0.0,0.0,0.0,36.0,0.0,0.0,26.0,0.0,0.0,...,7.0,0.0,1.0,0.0,7.0,10.0,75.0,105.0,66.0,4.0
2017-01-01 04:00:00-05:00,-05:00,0.0,0.0,0.0,34.0,0.0,0.0,33.0,0.0,0.0,...,4.0,1.0,0.0,0.0,14.0,28.0,97.0,131.0,91.0,2.0


## 4. Export Data

In [10]:
PATH = '../data/' # Modify this to fit your data folder

In [11]:
yellow.to_csv(PATH + 'yellow.csv', index=True)

Now `yellow` is ready to be used by the `NYC_Yellow_Cabs_Main.ipynb` notebook.